In [3]:

import random
import tkinter as tk

class Calendario:
    def __init__(self, num_equipes, max_confrontos_por_dia, times_importantes, max_partidas_por_dia_por_time):
        self.num_equipes = num_equipes
        self.max_confrontos_por_dia = max_confrontos_por_dia
        self.times_importantes = times_importantes
        self.max_partidas_por_dia_por_time = max_partidas_por_dia_por_time

        # if para tratar caso da num_equipes >= 2*max_confronto_por dia : alterar max_confrontos com divisão //2
        if not self.num_equipes>=2*self.max_confrontos_por_dia:

            self.max_confrontos_por_dia  = self.max_confrontos_por_dia//2
            print("Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia")

        # verificar se o max_partidas_por_dia_por_time é menor que o numero de equipes
        if not self.max_partidas_por_dia_por_time < self.num_equipes:
            self.max_partidas_por_dia_por_time = self.num_equipes - 1
            print("Alteração no numero de partidas por time por dia para atender a condição de max_partidas_por_dia_por_time < num_equipes")

        
        self.grade = self.gerar_grade_inicial()


    
    def gerar_grade_inicial(self):
        # Gera um calendário inicial válido
        grade = []
        for _ in range(self.num_equipes // 2):
            dia = []
            for _ in range(self.max_confrontos_por_dia):
                time1, time2 = random.sample(range(self.num_equipes), 2)
                dia.append((time1, time2))
            grade.append(dia)
        return grade

    def cruzar(self, outro):
        # Realiza o crossover entre dois calendários
        ponto_corte = random.randint(0, len(self.grade))
        filho1_grade = self.grade[:ponto_corte] + outro.grade[ponto_corte:]
        filho2_grade = outro.grade[:ponto_corte] + self.grade[ponto_corte:]

        # Corrigir filhos para respeitar max_partidas_por_dia_por_time
        filho1_grade = self.corrigir_max_partidas_por_dia(filho1_grade)
        filho2_grade = self.corrigir_max_partidas_por_dia(filho2_grade)

        # Criar novos objetos Calendário com as grades corrigidas
        filho1 = Calendario(self.num_equipes, self.max_confrontos_por_dia, self.times_importantes, self.max_partidas_por_dia_por_time)
        filho1.grade = filho1_grade

        filho2 = Calendario(self.num_equipes, self.max_confrontos_por_dia, self.times_importantes, self.max_partidas_por_dia_por_time)
        filho2.grade = filho2_grade

        return filho1, filho2
    
  
    

    def corrigir_max_partidas_por_dia(self, grade):
        # Corrige o calendário para respeitar o limite de partidas por time por dia
        partidas_por_dia_por_time = [{} for _ in grade]  # Lista de dicionários para cada dia

        for dia_idx, dia in enumerate(grade):
            partidas_por_time = partidas_por_dia_por_time[dia_idx]
            novos_confrontos_dia = []
            for confronto in dia:
                time1, time2 = confronto
                if partidas_por_time.get(time1, 0) <= self.max_partidas_por_dia_por_time and partidas_por_time.get(time2, 0) <= self.max_partidas_por_dia_por_time:
                    novos_confrontos_dia.append(confronto)
                    partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
                    partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
                    
                else:
                   
                    # Realocar confronto para outro dia
                    realocado = False
                    for novo_dia_idx in range(len(grade)):
                        if novo_dia_idx == dia_idx:
                            continue
                        novo_partidas_por_time = partidas_por_dia_por_time[novo_dia_idx]
                        if len(grade[novo_dia_idx]) < self.max_confrontos_por_dia and \
                                novo_partidas_por_time.get(time1, 0) < self.max_partidas_por_dia_por_time and \
                                novo_partidas_por_time.get(time2, 0) < self.max_partidas_por_dia_por_time:
                            grade[novo_dia_idx].append(confronto)
                            novo_partidas_por_time[time1] = novo_partidas_por_time.get(time1, 0) + 1
                            novo_partidas_por_time[time2] = novo_partidas_por_time.get(time2, 0) + 1
                            realocado = True
                            break
                    if not realocado:
                        # Criar um novo dia se necessário
                        grade.append([confronto])
                        partidas_por_dia_por_time.append({time1: 1, time2: 1})
            grade[dia_idx] = novos_confrontos_dia
            

        # Remover dias vazios
        grade = [dia for dia in grade if dia]
        return grade

    def mutar(self):
        # Seleciona dois dias aleatórios para trocar confrontos
        dia1_idx, dia2_idx = random.sample(range(len(self.grade)), 2)
        dia1 = self.grade[dia1_idx]
        dia2 = self.grade[dia2_idx]

        if not dia1 or not dia2:
            return  # Não é possível trocar se um dos dias estiver vazio

        # Seleciona confrontos aleatórios de cada dia
        confronto1 = random.choice(dia1)
        confronto2 = random.choice(dia2)

        # Cria cópias temporárias dos dias para testar a troca
        dia1_temp = dia1.copy()
        dia2_temp = dia2.copy()
        dia1_temp.remove(confronto1)
        dia2_temp.remove(confronto2)
        dia1_temp.append(confronto2)
        dia2_temp.append(confronto1)

        # Verifica se a troca respeita o limite de partidas por time por dia
        if self.verificar_limite_partidas(dia1_temp) and self.verificar_limite_partidas(dia2_temp):
            # Aplica a mutação se for válida
            self.grade[dia1_idx] = dia1_temp
            self.grade[dia2_idx] = dia2_temp

    def verificar_limite_partidas(self, dia):
        partidas_por_time = {}
        for confronto in dia:
            time1, time2 = confronto
            partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
            partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
        # Verifica se algum time excede o limite
        return all(count <= self.max_partidas_por_dia_por_time for count in partidas_por_time.values())

    def avaliar(self):
        # Avalia a qualidade do calendário, penalizando violações
        penalidade = 0
        # Penaliza confrontos importantes não alocados corretamente
        for (time1, time2), rodadas in self.times_importantes.items():
            presente = False
            for rodada in rodadas:
                if rodada - 1 < len(self.grade) and (time1, time2) in self.grade[rodada - 1]:
                    presente = True
                    break
            if not presente:
                penalidade += 10  # Penaliza fortemente

        # Penaliza times que jogam mais do que o permitido por dia
        for dia in self.grade:
            partidas_por_time = {}
            for confronto in dia:
                time1, time2 = confronto
                partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
                partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
            for count in partidas_por_time.values():
                if count > self.max_partidas_por_dia_por_time:
                    penalidade += (count - self.max_partidas_por_dia_por_time) * 5  # Penalidade proporcional

        return len(self.grade) + penalidade

    def verificar_calendario_valido(self):
        # Verifica se o calendário respeita o limite de partidas por dia
        for dia in self.grade:
            partidas_por_time = {}
            for confronto in dia:
                time1, time2 = confronto
                partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
                partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
            if any(count > self.max_partidas_por_dia_por_time for count in partidas_por_time.values()):
                return False
        return True
    

    def __str__(self) -> str:
        resultado = []
        for dia_num, confrontos in enumerate(self.grade):
            confrontos_str = ', '.join([f"{c[0]} vs {c[1]}" for c in confrontos])
            resultado.append(f"Dia {dia_num + 1}: {confrontos_str}")
        return '\n'.join(resultado)

    

def gerar_populacao_inicial(tamanho_populacao, num_equipes, max_confrontos_por_dia, times_importantes, max_partidas_por_dia_por_time):
    return [Calendario(num_equipes, max_confrontos_por_dia, times_importantes, max_partidas_por_dia_por_time) for _ in range(tamanho_populacao)]



def avaliar_populacao(populacao):
    return [individuo.avaliar() for individuo in populacao]

def selecao(populacao):
    # Seleção por torneio, pega 3 indivíduos aleatórios e escolhe os 2 melhores
    torneio = random.sample(populacao, 3)
    torneio.sort(key=lambda x: x.avaliar())
    return torneio[0], torneio[1]

def crossover(pai1, pai2):
    return pai1.cruzar(pai2)

def mutacao(individuo):
    individuo.mutar()

def selecionar_sobreviventes(populacao_antiga, nova_populacao):
    # Elitismo: mantém os melhores indivíduos das duas populações
    populacao_combinada = populacao_antiga + nova_populacao
    populacao_combinada.sort(key=lambda x: x.avaliar())
    return populacao_combinada[:len(populacao_antiga)]

def obter_melhor_individuo(populacao):
    return min(populacao, key=lambda x: x.avaliar())

def somar(lista):
    total = 0
    for i in range(len(lista)):
        total += lista[i]
    
    return total


def parse_times_importantes(texto):
    try:
        times_importantes = {}
        texto = texto.strip().replace("(", "").replace(")", "")
        if texto:
            itens = texto.split(", ")
            for item in itens:
                chave, valor = item.split(":")
                chave = tuple(map(int, chave.split(",")))
                valor = list(map(int, valor.strip("[]").split(",")))
                times_importantes[chave] = valor
        return times_importantes
    except Exception as e:
        raise ValueError(f"Erro ao processar os times importantes: {str(e)}")


class InterfaceAG:
    def __init__(self, root):
        self.root = root
        self.root.title("Algoritmo Genético - Configurações")

        tk.Label(root, text="Número de Equipes:").grid(row=0, column=0)
        self.num_equipes = tk.Entry(root)
        self.num_equipes.grid(row=0, column=1)

        tk.Label(root, text="Máx. Confrontos por Dia:").grid(row=1, column=0)
        self.max_confrontos_por_dia = tk.Entry(root)
        self.max_confrontos_por_dia.grid(row=1, column=1)

        tk.Label(root, text="Máx. Partidas por Dia por Time:").grid(row=2, column=0)
        self.max_partidas_por_dia_por_time = tk.Entry(root)
        self.max_partidas_por_dia_por_time.grid(row=2, column=1)

        tk.Label(root, text="Tamanho da População:").grid(row=3, column=0)
        self.tamanho_populacao = tk.Entry(root)
        self.tamanho_populacao.grid(row=3, column=1)

        tk.Label(root, text="Número de Gerações:").grid(row=4, column=0)
        self.num_geracoes = tk.Entry(root)
        self.num_geracoes.grid(row=4, column=1)

        tk.Label(root, text="Taxa de Crossover:").grid(row=5, column=0)
        self.taxa_crossover = tk.Entry(root)
        self.taxa_crossover.grid(row=5, column=1)

        tk.Label(root, text="Taxa de Mutação:").grid(row=6, column=0)
        self.taxa_mutacao = tk.Entry(root)
        self.taxa_mutacao.grid(row=6, column=1)

        tk.Label(root, text="Times Importantes:").grid(row=7, column=0)
        self.times_importantes = tk.Entry(root)
        self.times_importantes.grid(row=7, column=1)

        tk.Label(root, text="Turno Volta:").grid(row=8, column=0)
        self.turno_volta = tk.Entry(root)
        self.turno_volta.grid(row=8, column=1)

        tk.Button(root, text="Iniciar", command=self.iniciar_algoritmo).grid(row=9, column=0, columnspan=2)

        self.resultado = tk.Text(root, height=10, width=50)
        self.resultado.grid(row=10, column=0, columnspan=2)



    def iniciar_algoritmo(self):
        try:
            self.num_equipes = int(self.num_equipes.get())
            self.max_confrontos_por_dia = int(self.max_confrontos_por_dia.get())
            self.max_partidas_por_dia_por_time = int(self.max_partidas_por_dia_por_time.get())
            self.tamanho_populacao = int(self.tamanho_populacao.get())
            self.num_geracoes = int(self.num_geracoes.get())
            self.taxa_crossover = float(self.taxa_crossover.get())
            self.taxa_mutacao = float(self.taxa_mutacao.get())
            self.turno_volta = int(self.turno_volta.get())
            times_importantes_str = self.times_importantes.get().strip()
            if times_importantes_str:
                self.times_importantes = parse_times_importantes(times_importantes_str)
            else:
                self.times_importantes = {}
        except ValueError as e:
            message = f"Erro ao processar os parâmetros: {str(e)}"
            self.resultado.delete(1.0, tk.END)
            self.resultado.insert(tk.END, message + "\n")
            return

        self.t = 0
        self.parar_forcado = False
        self.populacao = gerar_populacao_inicial(self.tamanho_populacao, self.num_equipes, self.max_confrontos_por_dia, self.times_importantes, self.max_partidas_por_dia_por_time)
        avaliar_populacao(self.populacao)

        self.ultimos_5_avaliados = [0] * 5
        self.anterior = float('inf')
        self.contador = 0

        self.resultado.delete(1.0, tk.END)
        self.loop_algoritmo()

    def loop_algoritmo(self):
        if self.t < self.num_geracoes and not self.parar_forcado:
            self.t += 1
            nova_populacao = []

            for _ in range(self.tamanho_populacao // 2):
                pai1, pai2 = selecao(self.populacao)
                if random.random() < self.taxa_crossover:
                    filho1, filho2 = crossover(pai1, pai2)
                else:
                    filho1, filho2 = pai1, pai2

                if random.random() < self.taxa_mutacao:
                    mutacao(filho1)
                if random.random() < self.taxa_mutacao:
                    mutacao(filho2)

                nova_populacao.append(filho1)
                nova_populacao.append(filho2)

            x_total = avaliar_populacao(nova_populacao)
            self.populacao = selecionar_sobreviventes(self.populacao, nova_populacao)

            for x in x_total:
                self.ultimos_5_avaliados[self.contador % 5] = abs(x - self.anterior)
                self.anterior = x
                self.contador += 1
                soma = somar(self.ultimos_5_avaliados)

                if soma == 0:
                    self.resultado.insert(tk.END, f"Convergência detectada na geração {self.t}. Parando...\n")
                    self.parar_forcado = True
                    break

            self.resultado.insert(tk.END, f"Geração {self.t} avaliada.\n")

            # Agendar o próximo loop
            self.root.after(100, self.loop_algoritmo)
        else:
            melhor_solucao = obter_melhor_individuo(self.populacao)
            self.resultado.insert(tk.END, f"\nMelhor solução encontrada na geração {self.t}:\n")
            for dia, confrontos in enumerate(melhor_solucao.grade):
                self.resultado.insert(tk.END, f"Dia {dia + 1}:\n")
                self.resultado.insert(tk.END, f"| ")
                for plio in range(len(confrontos)):
                    buli = True
                    self.resultado.insert(tk.END, f"time {confrontos[plio][0]} X time {confrontos[plio][1]} | ")
                    if (plio+1)%2 == 0:
                        self.resultado.insert(tk.END, f"\n| ")
                        buli = False

                if buli:
                    self.resultado.insert(tk.END, f"\n")

                #printtar na tela   o resultado de cada dia de forma mais amigável  

            dia_anterior = dia
            if self.turno_volta == 1:
                for dia, confrontos in enumerate(melhor_solucao.grade):
                    self.resultado.insert(tk.END, f"Dia {dia + dia_anterior + 2}:\n")
                    self.resultado.insert(tk.END, f"| ")
                    for plio in range(len(confrontos)):
                        buli = True
                        self.resultado.insert(tk.END, f"time {confrontos[plio][1]} X time {confrontos[plio][0]} | ")
                        if (plio+1)%2 == 0:
                            self.resultado.insert(tk.END, f"\n| ")
                            buli = False

                    if buli:
                        self.resultado.insert(tk.END, f"\n")
 
def main():
    root = tk.Tk()
    app = InterfaceAG(root)
    root.mainloop()

if __name__ == "__main__":
    main()


Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confrontos por dia para atender a condição de num_equipes >= 2*max_confronto_por dia
Alteraçao no numero de confr

KeyboardInterrupt: 